 Seattle - Airbnb Data Set.



In order to get a better grip over the Airbnb data set. Lets wrangle to get a better understanding, and prepare the data set.


In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline

import numba
from numba import cuda

#### Loading the Datasets

In [33]:
airbnb_listing_df = pd.read_csv('Data/Airbnb_Seattle/seattle_airbnb_listing.csv')

airbnb_neighbourhood_df = pd.read_csv('Data/Airbnb_Seattle/seattle_airbnb_neighbourhoods.csv')

airbnb_neighbourhood_geojson_df = pd.read_csv('Data/Airbnb_Seattle/seattle_airbnb_neighbourhoods_geojson.csv')

airbnb_reviews_df = pd.read_csv('Data/Airbnb_Seattle/seattle_airbnb_reviews.csv')

#### Finding the Rows and Columns in the Dataframes

In [16]:
airbnb_listing_df.shape

(8740, 16)

In [17]:
airbnb_neighbourhood_df.shape

(90, 2)

In [31]:
airbnb_neighbourhood_geojson_df.shape

(90, 2)

In [32]:
airbnb_reviews_df.shape

(373096, 2)

#### Examning the dataframes

In [25]:
airbnb_listing_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2318,"Casa Madrona - Urban Oasis, 1 block from the P...",2536,Megan,Central Area,Madrona,47.610819,-122.290816,Entire home/apt,296,4,20,9/18/2018,0.16,3,345
1,4291,Sunrise in Seattle Master Suite,35749,Jess & Joey,Other neighborhoods,Roosevelt,47.687801,-122.313427,Private room,82,2,63,10/6/2018,0.96,5,365
2,5682,"Cozy Studio, min. to downtown -WiFi",8993,Maddy,Delridge,South Delridge,47.523979,-122.359891,Entire home/apt,48,3,461,11/2/2018,4.37,1,24
3,6606,"Fab, private seattle urban cottage!",14942,Joyce,Other neighborhoods,Wallingford,47.654109,-122.337605,Entire home/apt,90,2,134,10/21/2018,1.18,3,287
4,9419,Glorious sun room w/ memory foambed,30559,Angielena,Other neighborhoods,Georgetown,47.550620,-122.320135,Private room,65,2,130,9/9/2018,1.29,5,336


In [27]:
airbnb_neighbourhood_df.head()

,neighbourhood_group,neighbourhood
0,Ballard,Adams
1,Ballard,Loyal Heights
2,Ballard,Sunset Hill
3,Ballard,West Woodland
4,Ballard,Whittier Heights


In [29]:
airbnb_neighbourhood_geojson_df.head()

,neighbourhood_group,neighbourhood
0,Ballard,Adams
1,Ballard,Loyal Heights
2,Ballard,Sunset Hill
3,Ballard,West Woodland
4,Ballard,Whittier Heights


In [30]:
airbnb_reviews_df.head()

,listing_id,date
0,2318,2008-09-15
1,2318,2017-01-10
2,2318,2017-04-01
3,2318,2017-04-25
4,2318,2017-06-18


#### Testing Numba Library for GPU processing

In [20]:
print(cuda.gpus)

<Managed Device 0>


In [24]:
#thread id in 1D block

# blockwidth = number of threads per block
bw = cuda.blockDim
print(bw)


numba.cuda.stubs.blockDim